# Importando as Bibliotecas

In [65]:
import pandas as pd
import imaplib
import email
import yaml  
import quopri
import re
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials


In [66]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Extraindo as Informações do Email


In [67]:
with open("credentials.yml") as f:  # Abre o arquivo "credentials.yml" para leitura
    content = f.read()


my_credentials = yaml.load(content, Loader=yaml.FullLoader) # Importar o nome de usuário e a senha do arquivo YAML

user, password = my_credentials["user"], my_credentials["password"] # Carrega o nome do usuário e a senha do arquivo YAML

imap_url = 'imap.gmail.com' # URL para conexão IMAP

my_mail = imaplib.IMAP4_SSL(imap_url) # Conexão com o GMAIL usando SSL
my_mail.login(user, password) # Fazer login usando as credenciais
my_mail.select('Inbox') # Selecionar a Caixa de Entrada para buscar mensagens


key = 'FROM' # Definir a chave para a pesquisa de e-mail
value = 'lucas.rossi@videosoft.com.br' # Definir o valor para a pesquisa de e-mail

_, data = my_mail.search(None, '(UNSEEN)', key, value)  # Buscar e-mails com chave e valor específicos


mail_id_list = data[0].split() # IDs de todos os e-mails que queremos buscar

msgs = [] # Lista vazia para armazenar todas as mensagens
for num in mail_id_list: # Iterar pelas mensagens e extrair os dados para a lista msgs
    typ, data = my_mail.fetch(num, '(RFC822)') # RFC822 retorna a mensagem inteira (BODY busca apenas o corpo)
    msgs.append(data)


email_results = []
for msg in msgs[::-1]: # Iterar pelas mensagens para extrair informações
    for response_part in msg:
        if type(response_part) is tuple:
            my_msg = email.message_from_bytes((response_part[1]))
            email_info = {
                "subject": my_msg['subject'],
                "from": my_msg['from'],
                "body": "" # Inicialmente, o corpo do e-mail é definido como vazio
            }
            for part in my_msg.walk():  
                if part.get_content_type() == 'text/plain':  # Se for texto simples, atribuir o corpo ao dicionário
                    email_info["body"] = part.get_payload()
            email_results.append(email_info)  # Adicionar o dicionário de informações do e-mail à lista

# Transformando o Dado


In [68]:
data_json = json.dumps([email_results], separators = (',',':')) # converter a lista email_results em formato JSON

In [69]:
def decode(value):  # Função para decodificar uma string codificada no formato Quoted-Printable
    decoded_bytes = quopri.decodestring(value.encode('utf-8'))  # Decodifica os bytes codificados usando Quoted-Printable para bytes decodificados
    decoded_text =  decoded_bytes.decode('utf-8')  # Decodifica os bytes decodificados em uma string de texto
    return decoded_text  # Retorna a string de texto decodificada

In [70]:
data_decode = decode(data_json) # Aplicando a função na variável data_json

In [71]:
data = json.loads(data_decode) # Carrega os dados JSON decodificados em uma estrutura de dados Python

In [72]:
data_dic = data[0] # Acessar o primeiro elemento da lista de dados e armazená-lo em data_dic

In [73]:
df = pd.DataFrame(data_dic) # Transforma o Dicionário data_dic em DataFrame e salva na variável df 

In [74]:

pattern_cnpj = r"\d{14}" # Expressão regular para encontrar CNPJ com 14 dígitos numéricos
pattern_phone = r"\(\d{2}\)\d{9}" # Expressão regular para encontrar número de telefone no formato (XX)XXXXXXXXX
pattern_social = r"\*Razão social : \*(.*?)\r\n\*Nome Fantasia:" # Expressão regular para extrair a "Razão Social" de um texto
pattern_email = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b" # Expressão regular para encontrar endereços de e-mail válidos
pattern_description = r'Totem Auto atendimento \(piso 27"\) LOCAÇÃO' # Expressão regular para encontrar uma descrição específica
pattern_description2 = r"Totem Auto atendimento \(balcão vertical\) LOCAÇÃO" # Expressão regular para encontrar outra descrição específica
pattern_quantity = r"Não se aplica (\d+,\d+)" # Expressão regular para encontrar uma quantidade específica
pattern_value = r"BRL (\d+,\d+)" # Expressão regular para encontrar um valor no formato "BRL X,Y"


In [75]:
df['cnpjs'] = df['body'].apply(lambda x: re.findall(pattern_cnpj, x)) # Aplicar a expressão regular pattern_cnpj à coluna 'body' e criar uma nova coluna 'cnpjs'
df['phone'] = df['body'].apply(lambda x: re.findall(pattern_phone, x)) # Aplicar a expressão regular pattern_phone à coluna 'body' e criar uma nova coluna 'phone'
df['social'] = df['body'].apply(lambda x: re.findall(pattern_social, x)) # Aplicar a expressão regular pattern_social à coluna 'body' e criar uma nova coluna 'social'
df['email'] = df['body'].apply(lambda x: re.findall(pattern_email, x)) # Aplicar a expressão regular pattern_email à coluna 'body' e criar uma nova coluna 'email'
df['description1'] = df['body'].apply(lambda x: re.findall(pattern_description, x)) # Aplicar a expressão regular pattern_description à coluna 'body' e criar uma nova coluna 'description1'
df['description2'] = df['body'].apply(lambda x: re.findall(pattern_description2, x)) # Aplicar a expressão regular pattern_description à coluna 'body' e criar uma nova coluna 'description2'
df['quantity'] = df['body'].apply(lambda x: re.findall(pattern_quantity, x)) # Aplicar a expressão regular pattern_description à coluna 'body' e criar uma nova coluna 'qauntity'
df['value'] = df['body'].apply(lambda x: re.findall(pattern_value, x)) # Aplicar a expressão regular pattern_value à coluna 'body' e criar uma nova coluna 'value'





In [76]:
drop = ['subject','from', 'body']  # Lista de colunas a serem removidas do DataFrame

In [77]:
df = df.drop(drop, axis=1) # Remover as colunas listadas na lista 'drop' do DataFrame 'df'

In [78]:
df['cnpjs'] = df['cnpjs'].apply(lambda x: x[1] if len(x) > 1 else None) # Aplicar uma função para selecionar o segundo valor em 'cnpjs' se houver mais de um valor, caso contrário, atribuir None
df['email'] = df['email'].apply(lambda x: x[2] if len(x) > 2 else None) # Aplicar uma função para selecionar o terceiro valor em 'email' se houver mais de um valor, caso contrário, atribuir None
df['value'] = df['value'].apply(lambda x: x[0] if len(x) > 1 else None) # Aplicar uma função para selecionar o primeiro valor em 'value' se houver mais de um valor, caso contrário, atribuir None


In [79]:
df = df.applymap(lambda x: str(x).strip('[]') if isinstance(x, list) else x) # Aplicar uma função para remover os colchetes de listas e espaços em branco das células
df = df.applymap(lambda x: str(x).strip('') if isinstance(x, list) else x) # Aplicar uma função para remover espaços em branco vazios das células
df = df.applymap(lambda x: x.replace(r"'", '') if isinstance(x, str) else x) # Aplicar uma função para remover aspas simples de strings nas células


In [80]:
df['description'] = df['description1'].fillna('') + df['description2'].fillna('') # Criar uma nova coluna 'description' que combina os valores de 'description1' e 'description2' se algum valor estiver faltando(NaN), é preenchido com uma string vazia


In [81]:
drop_description = ['description1', 'description2'] # Lista de colunas a serem removidas do DataFrame

In [82]:
df = df.drop(drop_description, axis = 1) # Remover as colunas listadas na lista 'drop_description' do DataFrame 'df'

In [83]:
df = df[['cnpjs', 'phone', 'social', 'email', 'quantity', 'description', 'value']] # Reorganizar as colunas do DataFrame 'df' na ordem especificada

# Enviando para Google Sheets

In [84]:

credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials_google_sheets.json') # Carregar as credenciais do arquivo JSON para autenticação no Google Sheets
client = gspread.authorize(credentials) # Autorizar o cliente usando as credenciais carregadas

In [85]:
spreadsheet = client.open('Test Email') # Autorizar o cliente usando as credenciais carregadas

In [86]:
index = 0 # Definir o índice da linha que deseja obter os dados
data_to_send = df.iloc[index].values.tolist() # Obter os valores da linha no índice especificado como uma lista

In [87]:
worksheet = spreadsheet.get_worksheet(0) # Obter a primeira planilha da planilha 'Test Email'

In [88]:
for i, row in df.iterrows(): # Iterar sobre cada linha do DataFrame 'df'
    data_to_send = row.values.tolist() # Obter os valores da linha atual como uma lista
    worksheet.append_row(data_to_send)  # Adicionar uma nova linha à planilha 'worksheet' com os valores da linha atual